# **STEP 11:** Uniformize Mask Sizes

In [10]:
#the python debugger is a library that provides an interactive source code debugger for Python programs
import pdb

import SimpleITK as sitk
import os
import shutil

### Copy the patients for a new folder

In [11]:
#### CT Images - Isodose Masks, brain masks and tumor masks

In [16]:
folder_1 = "/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More"
folder_2 = "/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More"
new_folder = "/notebooks/disk2/RT_Dataset_2/06_Processed_Patients"

os.makedirs(new_folder, exist_ok=True)

for item in os.listdir(folder_1):
    s = os.path.join(folder_1, item)
    d = os.path.join(new_folder, item)
    if os.path.isdir(s):
        shutil.copytree(s, d, dirs_exist_ok=True)
    else:
        shutil.copy2(s, d)

for item in os.listdir(folder_2):
    s = os.path.join(folder_2, item)
    d = os.path.join(new_folder, item)
    if os.path.isdir(s):
        shutil.copytree(s, d, dirs_exist_ok=True)
    else:
        shutil.copy2(s, d)


In [17]:
#### RM Images

In [18]:
rm_folder = "/notebooks/disk2/RT_Dataset_2/05_Dataset_Registered"
target_folder = "/notebooks/disk2/RT_Dataset_2/06_Processed_Patients"

# List subfolders in RM folder
rm_subfolders = [d for d in os.listdir(rm_folder) if os.path.isdir(os.path.join(rm_folder, d))]

for subfolder in rm_subfolders:
    rm_subfolder_path = os.path.join(rm_folder, subfolder)
    target_subfolder_path = os.path.join(target_folder, subfolder)

    # Check if subfolder with the same name exists in target folder
    if os.path.exists(target_subfolder_path) and os.path.isdir(target_subfolder_path):
        print(f"Copying contents from '{rm_subfolder_path}' to '{target_subfolder_path}'")

        # Copy all files and folders inside rm_subfolder_path into target_subfolder_path
        for item in os.listdir(rm_subfolder_path):
            src_item = os.path.join(rm_subfolder_path, item)
            dst_item = os.path.join(target_subfolder_path, item)

            if os.path.isdir(src_item):
                if os.path.exists(dst_item):
                    print(f"Folder '{dst_item}' already exists, skipping copy of this folder.")
                else:
                    print(f"Copying folder '{src_item}' to '{dst_item}'")
                    shutil.copytree(src_item, dst_item)
            else:
                # It's a file
                if os.path.exists(dst_item):
                    print(f"File '{dst_item}' already exists, skipping.")
                else:
                    print(f"Copying file '{src_item}' to '{dst_item}'")
                    shutil.copy2(src_item, dst_item)
    else:
        print(f"Subfolder '{subfolder}' does not exist in target folder, skipping.")

Copying contents from '/notebooks/disk2/RT_Dataset_2/05_Dataset_Registered/Patient_61' to '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61'
Folder '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_TRATAMENTO' already exists, skipping copy of this folder.
File '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/.DS_Store' already exists, skipping.
Folder '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_FOLLOW_UP' already exists, skipping copy of this folder.
Copying contents from '/notebooks/disk2/RT_Dataset_2/05_Dataset_Registered/Patient_78' to '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_78'
File '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_78/.DS_Store' already exists, skipping.
Folder '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_78/P78_RM_1_TRATAMENTO' already exists, skipping copy of this folder.
Folder '/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patie

### Resize Brain Masks to Treatment Images

In [3]:
# Path to processed patients
base_path = "/notebooks/disk2/RT_Dataset_2/06_Processed_Patients"

# Mask priority
mask_keywords = ["Whole-Brain", "Skull", "Outer-Contour"]

print("Resampling Brain Masks to Treatment Images")

# Loop through patient folders
for patient_folder in os.listdir(base_path):
    if not patient_folder.startswith("Patient_"):
        continue
    
    patient = patient_folder.split("_")[1]
    print(f"\nProcessing Patient_{patient}")

    # Set folder paths
    patient_path = os.path.join(base_path, f"Patient_{patient}")
    ct_folder = os.path.join(patient_path, f"P{patient}_CT_1_TRATAMENTO")
    treatment_folder = os.path.join(patient_path, f"P{patient}_RM_1_TRATAMENTO")
    #print(follow_up_folder)
    output_mask_path = os.path.join(ct_folder, f"P{patient}_brain_mask_treat.nii.gz")

    # 1. Find the brain mask
    brain_mask_path = None
    if os.path.exists(ct_folder):
        for fname in os.listdir(ct_folder):
            for keyword in mask_keywords:
                if keyword in fname and fname.endswith(".nii.gz"):
                    brain_mask_path = os.path.join(ct_folder, fname)
                    print(brain_mask_path)
                    break
            if brain_mask_path:
                break

    if not brain_mask_path or not os.path.exists(brain_mask_path):
        print(f"No valid brain mask found for Patient_{patient}, skipping.")
        continue

    # 2. Find the follow-up image
    treatment_path = None
    if os.path.exists(treatment_folder):
        for root, _, files in os.walk(treatment_folder):
            for file in files:
                if file.endswith(".nii.gz"):
                    treatment_path = os.path.join(root, file)
                    break
            if treatment_path:
                break

    if not treatment_path or not os.path.exists(treatment_path):
        print(f"No treatment image found for Patient_{patient}, skipping.")
        continue

    # 3. Resample the brain mask to match the follow-up image
    try:
        brain_mask = sitk.ReadImage(brain_mask_path)
        treatment_image = sitk.ReadImage(treatment_path)

        resampler = sitk.ResampleImageFilter()
        resampler.SetReferenceImage(treatment_image)
        resampler.SetInterpolator(sitk.sitkNearestNeighbor)
        resampler.SetOutputOrigin(treatment_image.GetOrigin())
        resampler.SetOutputSpacing(treatment_image.GetSpacing())
        resampler.SetOutputDirection(treatment_image.GetDirection())
        resampler.SetSize(treatment_image.GetSize())

        resampled_mask = resampler.Execute(brain_mask)
        sitk.WriteImage(resampled_mask, output_mask_path)

        print(f"✓ Done for Patient_{patient}")

    except Exception as e:
        print(f"Error processing Patient_{patient}: {e}")


Resampling Brain Masks to Treatment Images

Processing Patient_61
/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_CT_1_TRATAMENTO/mask_Outer-Contour.nii.gz
✓ Done for Patient_61

Processing Patient_78
/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_78/P78_CT_1_TRATAMENTO/mask_Skull.nii.gz
✓ Done for Patient_78

Processing Patient_94
/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_94/P94_CT_1_TRATAMENTO/mask_Skull.nii.gz
✓ Done for Patient_94

Processing Patient_71
/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_71/P71_CT_1_TRATAMENTO/mask_Outer-Contour.nii.gz
✓ Done for Patient_71

Processing Patient_73
/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_73/P73_CT_1_TRATAMENTO/mask_Skull.nii.gz
✓ Done for Patient_73

Processing Patient_90
/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_90/P90_CT_1_TRATAMENTO/mask_Skull.nii.gz
✓ Done for Patient_90

Processing Patient_56
/notebooks/disk2/RT_Dataset_2/06_Processed

In [4]:
# Base path for patients
base_path = "/notebooks/disk2/RT_Dataset_2/06_Processed_Patients"

print("Organizing brain mask .nii.gz files into RM treatment folders...")

# Loop through patient folders
for patient_folder in os.listdir(base_path):
    if not patient_folder.startswith("Patient_"):
        continue

    patient = patient_folder.split("_")[1]
    ct_folder = os.path.join(base_path, patient_folder, f"P{patient}_CT_1_TRATAMENTO")
    rm_folder = os.path.join(base_path, patient_folder, f"P{patient}_RM_1_TRATAMENTO")

    if not os.path.exists(ct_folder):
        print(f"CT folder not found for Patient_{patient}, skipping.")
        continue

    if not os.path.exists(rm_folder):
        os.makedirs(rm_folder)
        print(f" Created RM folder for Patient_{patient}")

    for file in os.listdir(ct_folder):
        if file.endswith(".nii.gz") and "brain_mask_treat" in file:
            src_path = os.path.join(ct_folder, file)
            dst_path = os.path.join(rm_folder, file)

            try:
                shutil.move(src_path, dst_path)
                print(f"Moved: {file} → {rm_folder}")
            except Exception as e:
                print(f" Error moving {file}: {e}")

print("✅ All done.")

Organizing brain mask .nii.gz files into RM treatment folders...
Moved: P61_brain_mask_treat.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_TRATAMENTO
Moved: P78_brain_mask_treat.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_78/P78_RM_1_TRATAMENTO
Moved: P94_brain_mask_treat.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_94/P94_RM_1_TRATAMENTO
Moved: P71_brain_mask_treat.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_71/P71_RM_1_TRATAMENTO
Moved: P73_brain_mask_treat.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_73/P73_RM_1_TRATAMENTO
Moved: P90_brain_mask_treat.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_90/P90_RM_1_TRATAMENTO
Moved: P56_brain_mask_treat.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_56/P56_RM_1_TRATAMENTO
Moved: P75_brain_mask_treat.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patien

### Resize Brain Masks to Follow Up Images

In [9]:
# Path to processed patients
base_path = "/notebooks/disk2/RT_Dataset_2/06_Processed_Patients"

# Mask priority
mask_keywords = ["Whole-Brain", "Skull", "Outer-Contour"]

print("Resampling Brain Masks to Follow Up Images")

# Loop through patient folders
for patient_folder in os.listdir(base_path):
    if not patient_folder.startswith("Patient_"):
        continue

    patient = patient_folder.split("_")[1]
    print(f"\nProcessing Patient_{patient}")
    
    # Set folder paths
    patient_path = os.path.join(base_path, f"Patient_{patient}")
    ct_folder = os.path.join(patient_path, f"P{patient}_CT_1_TRATAMENTO")
    follow_up_folder = os.path.join(patient_path, f"P{patient}_RM_1_FOLLOW_UP")
    #print(follow_up_folder)
    output_mask_path = os.path.join(ct_folder, f"P{patient}_brain_mask.nii.gz")
    
    # 1. Find the brain mask
    brain_mask_path = None
    if os.path.exists(ct_folder):
        for fname in os.listdir(ct_folder):
            for keyword in mask_keywords:
                if keyword in fname and fname.endswith(".nii.gz"):
                    brain_mask_path = os.path.join(ct_folder, fname)
                    print(brain_mask_path)
                    break
            if brain_mask_path:
                break
    
    if not brain_mask_path or not os.path.exists(brain_mask_path):
        print(f"No valid brain mask found for Patient_{patient}, skipping.")
        continue
    
    # 2. Find the follow-up image
    follow_up_image_path = None
    if os.path.exists(follow_up_folder):
        for root, _, files in os.walk(follow_up_folder):
            for file in files:
                if file.endswith(".nii.gz"):
                    follow_up_image_path = os.path.join(root, file)
                    break
            if follow_up_image_path:
                break
    
    if not follow_up_image_path or not os.path.exists(follow_up_image_path):
        print(f"No follow-up image found for Patient_{patient}, skipping.")
        continue
    
    # 3. Resample the brain mask to match the follow-up image
    try:
        brain_mask = sitk.ReadImage(brain_mask_path)
        follow_up_image = sitk.ReadImage(follow_up_image_path)
    
        resampler = sitk.ResampleImageFilter()
        resampler.SetReferenceImage(follow_up_image)
        resampler.SetInterpolator(sitk.sitkNearestNeighbor)
        resampler.SetOutputOrigin(follow_up_image.GetOrigin())
        resampler.SetOutputSpacing(follow_up_image.GetSpacing())
        resampler.SetOutputDirection(follow_up_image.GetDirection())
        resampler.SetSize(follow_up_image.GetSize())
    
        resampled_mask = resampler.Execute(brain_mask)
        sitk.WriteImage(resampled_mask, output_mask_path)
    
        print(f"✓ Done for Patient_{patient}")
    
    except Exception as e:
        print(f"Error processing Patient_{patient}: {e}")


Resampling Brain Masks to Follow Up Images

Processing Patient_55
/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_55/P55_CT_1_TRATAMENTO/mask_Whole-Brain.nii.gz
✓ Done for Patient_55


In [28]:
# Base path for patients
base_path = "/notebooks/disk2/RT_Dataset_2/06_Processed_Patients"

print("Organizing brain mask .nii.gz files into RM follow-up folders...")

# Loop through patient folders
for patient_folder in os.listdir(base_path):
    if not patient_folder.startswith("Patient_"):
        continue

    patient = patient_folder.split("_")[1]
    ct_folder = os.path.join(base_path, patient_folder, f"P{patient}_CT_1_TRATAMENTO")
    rm_folder = os.path.join(base_path, patient_folder, f"P{patient}_RM_1_FOLLOW_UP")

    if not os.path.exists(ct_folder):
        print(f"CT folder not found for Patient_{patient}, skipping.")
        continue

    if not os.path.exists(rm_folder):
        os.makedirs(rm_folder)
        print(f" Created RM folder for Patient_{patient}")

    for file in os.listdir(ct_folder):
        if file.endswith(".nii.gz") and "brain_mask" in file and "brain_mask_ct" not in file:
            src_path = os.path.join(ct_folder, file)
            dst_path = os.path.join(rm_folder, file)

            try:
                shutil.move(src_path, dst_path)
                print(f"Moved: {file} → {rm_folder}")
            except Exception as e:
                print(f" Error moving {file}: {e}")

print("✅ All done.")


Organizing brain mask .nii.gz files into RM follow-up folders...
Moved: P61_brain_mask.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_FOLLOW_UP
Moved: P78_brain_mask.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_78/P78_RM_1_FOLLOW_UP
Moved: P94_brain_mask.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_94/P94_RM_1_FOLLOW_UP
Moved: P71_brain_mask.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_71/P71_RM_1_FOLLOW_UP
Moved: P73_brain_mask.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_73/P73_RM_1_FOLLOW_UP
Moved: P90_brain_mask.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_90/P90_RM_1_FOLLOW_UP
 Created RM folder for Patient_93
Moved: P56_brain_mask.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_56/P56_RM_1_FOLLOW_UP
Moved: P75_brain_mask.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_75/P75_RM_1_FOLLOW_

### Resize Brain Masks to CT Images

In [8]:
# Path to processed patients
base_path = "/notebooks/disk2/RT_Dataset_2/06_Processed_Patients"

# Mask priority
mask_keywords = ["Whole-Brain", "Skull", "Outer-Contour"]

print("Resampling Brain Masks to CT Images")

# Loop through patient folders
for patient_folder in os.listdir(base_path):
    if not patient_folder.startswith("Patient_"):
        continue

    patient = patient_folder.split("_")[1]
    print(f"Processing Patient_{patient}")

    # Set folder paths
    patient_path = os.path.join(base_path, f"Patient_{patient}")
    ct_folder = os.path.join(patient_path, f"P{patient}_CT_1_TRATAMENTO")
    output_mask_path = os.path.join(ct_folder, f"P{patient}_brain_mask_ct.nii.gz")

    # 1. Find the brain mask
    brain_mask_path = None
    if os.path.exists(ct_folder):
        for fname in os.listdir(ct_folder):
            for keyword in mask_keywords:
                if keyword in fname and fname.endswith(".nii.gz"):
                    brain_mask_path = os.path.join(ct_folder, fname)
                    print(brain_mask_path)
                    break
            if brain_mask_path:
                break

    if not brain_mask_path or not os.path.exists(brain_mask_path):
        print(f"No valid brain mask found for Patient_{patient}, skipping.")
        continue

    # 2. Find the ct image
    ct_image_path = None
    for fname in os.listdir(ct_folder):
        if "Windowed" in fname and fname.endswith(".nii.gz"):
            ct_image_path = os.path.join(ct_folder, fname)
            print(f"CT image found: {fname}")
            break

    if not ct_image_path or not os.path.exists(ct_image_path):
        print(f"No CT image found for Patient_{patient}, skipping.")
        continue

    # 3. Resample the brain mask to match the follow-up image
    try:
        brain_mask = sitk.ReadImage(brain_mask_path)
        ct_image = sitk.ReadImage(ct_image_path)

        resampler = sitk.ResampleImageFilter()
        resampler.SetReferenceImage(ct_image)
        resampler.SetInterpolator(sitk.sitkNearestNeighbor)
        resampler.SetOutputOrigin(ct_image.GetOrigin())
        resampler.SetOutputSpacing(ct_image.GetSpacing())
        resampler.SetOutputDirection(ct_image.GetDirection())
        resampler.SetSize(ct_image.GetSize())

        resampled_mask = resampler.Execute(brain_mask)
        sitk.WriteImage(resampled_mask, output_mask_path)

        print(f"✓ Done for Patient_{patient}")

    except Exception as e:
        print(f"Error processing Patient_{patient}: {e}")


Resampling Brain Masks to CT Images
Processing Patient_61
/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_CT_1_TRATAMENTO/mask_Outer-Contour.nii.gz
CT image found: P61_CT_1_TRATAMENTO_Windowed.nii.gz
✓ Done for Patient_61
Processing Patient_78
/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_78/P78_CT_1_TRATAMENTO/mask_Skull.nii.gz
CT image found: P78_CT_1_TRATAMENTO_Windowed_Denoised.nii.gz
✓ Done for Patient_78
Processing Patient_94
/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_94/P94_CT_1_TRATAMENTO/mask_Skull.nii.gz
CT image found: P94_CT_1_TRATAMENTO_Windowed_Denoised.nii.gz
✓ Done for Patient_94
Processing Patient_71
/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_71/P71_CT_1_TRATAMENTO/mask_Outer-Contour.nii.gz
CT image found: P71_CT_1_TRATAMENTO_Windowed_Denoised.nii.gz
✓ Done for Patient_71
Processing Patient_73
/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_73/P73_CT_1_TRATAMENTO/mask_Skull.nii.gz
CT image f

### Resize Tumor Individual Masks to Treatment Images

In [33]:
base_path = "/notebooks/disk2/RT_Dataset_2/06_Processed_Patients"

print("Deleting all CT 'treat' .nii.gz files...")

# Loop through patient folders
for patient_folder in os.listdir(base_path):
    if not patient_folder.startswith("Patient_"):
        continue

    patient = patient_folder.split("_")[1]
    ct_folder = os.path.join(base_path, patient_folder, f"P{patient}_CT_1_TRATAMENTO")

    if not os.path.exists(ct_folder):
        print(f"⚠ CT folder not found for Patient_{patient}, skipping.")
        continue

    # Go through files and delete matching ones
    for file in os.listdir(ct_folder):
        if file.endswith(".nii.gz") and "treat_ct" in file:
            file_path = os.path.join(ct_folder, file)
            try:
                os.remove(file_path)
                print(f"Deleted: {file_path}")
            except Exception as e:
                print(f"Could not delete {file_path}: {e}")

print("✅ Done.")

Deleting all CT 'treat' .nii.gz files...
Deleted: /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_CT_1_TRATAMENTO/mask_Brain_treat_ct.nii.gz
Deleted: /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_CT_1_TRATAMENTO/mask_Tumor_treat_ct.nii.gz
Deleted: /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_CT_1_TRATAMENTO/mask_PTV-3x8-opt_treat_fu_treat_ct.nii.gz
Deleted: /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_CT_1_TRATAMENTO/mask_Lens-Left_treat_ct.nii.gz
Deleted: /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_CT_1_TRATAMENTO/mask_Optic-Tract-Right_treat_ct.nii.gz
Deleted: /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_CT_1_TRATAMENTO/mask_PTV-3x8_treat_ct.nii.gz
Deleted: /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_CT_1_TRATAMENTO/mask_Optic-Tract-Left_treat_ct.nii.gz
Deleted: /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_CT_1_T

In [27]:
print("Resampling Tumor INDIVIDUAL Masks to Treatment Images")

base_path = "/notebooks/disk2/RT_Dataset_2/06_Processed_Patients"
mask_keywords = ["Whole-Brain", "Skull", "Outer-Contour"]

# Loop through patient folders
for patient_folder in os.listdir(base_path):
    if not patient_folder.startswith("Patient_"):
        continue

    patient = patient_folder.split("_")[1]
    print(f"Processing Patient_{patient}")

    # Paths
    patient_path = os.path.join(base_path, f"Patient_{patient}")
    ct_folder = os.path.join(patient_path, f"P{patient}_CT_1_TRATAMENTO")
    treatment_folder = os.path.join(patient_path, f"P{patient}_RM_1_TRATAMENTO")

    # Find TREATMENT image (one .nii.gz file inside treatment folder)
    mr_volume_image = None
    if os.path.exists(treatment_folder):
        for file in os.listdir(treatment_folder):
            if file.endswith(".nii.gz"):
                mr_volume_image = sitk.ReadImage(os.path.join(treatment_folder, file))
                break

    if mr_volume_image is None:
        print(f"No treatment image found for Patient_{patient}, skipping.")
        continue

    tumors_path = ct_folder

    # Resample all tumor masks
    for file in os.listdir(ct_folder):
        if file.startswith("mask_") and file.endswith(".nii.gz") and not any(keyword in file for keyword in mask_keywords):
            mask_name = file.removesuffix(".nii.gz")
            tumor_mask_path = os.path.join(tumors_path, file)
            tumor_mask_treat_path = os.path.join(tumors_path, mask_name + "_treat.nii.gz")

            tumor_mask = sitk.ReadImage(tumor_mask_path)

            # Resample to match MR volume
            resampler = sitk.ResampleImageFilter()
            resampler.SetReferenceImage(mr_volume_image)
            resampler.SetInterpolator(sitk.sitkNearestNeighbor)
            resampler.SetOutputOrigin(mr_volume_image.GetOrigin())
            resampler.SetOutputSpacing(mr_volume_image.GetSpacing())
            resampler.SetOutputDirection(mr_volume_image.GetDirection())
            resampler.SetSize(mr_volume_image.GetSize())

            resampled_mask = resampler.Execute(tumor_mask)
            sitk.WriteImage(resampled_mask, tumor_mask_treat_path)

            print(f"Resampled: {mask_name}")

    print(f"Done for Patient_{patient}")


Resampling Tumor INDIVIDUAL Masks to Treatment Images
Processing Patient_61
Resampled: mask_Lens-Right
Resampled: mask_Chiasm
Resampled: mask_PTV-3x8
Resampled: mask_PTV-3x8-1cm
Resampled: mask_Optic-Nerve-Right
Resampled: mask_Brainstem
Resampled: mask_Eye-Left
Resampled: mask_Tumor
Resampled: mask_Optic-Tract-Right
Resampled: mask_Brain
Resampled: mask_Eye-Right
Resampled: mask_Lens-Left
Resampled: mask_PTV-3x8-opt
Resampled: mask_Optic-Tract-Left
Resampled: mask_Optic-Nerve-Left
Done for Patient_61
Processing Patient_78
Resampled: mask_MtFrontEProf
Resampled: mask_Lens-Right
Resampled: mask_MetVerCerebelar
Resampled: mask_MerFrEsq
Resampled: mask_MetFDirInf
Resampled: mask_MetODir
Resampled: mask_MtFrMedAntEs
Resampled: mask_Chiasm
Resampled: mask_MetCapIntDir
Resampled: mask_MetIntHemisf
Resampled: mask_MetFPDIR
Resampled: mask_MetParProfD
Resampled: mask_Optic-Nerve-Right
Resampled: mask_MetOccEsqInf
Resampled: mask_MetMesencDir
Resampled: mask_Brainstem
Resampled: mask_Eye-Left
R

In [14]:
# Base path for patients
base_path = "/notebooks/disk2/RT_Dataset_2/06_Processed_Patients"
patients = ["Patient_54", "Patient_59","Patient_67", "Patient_68", "Patient_69", "Patient_70","Patient_82", "Patient_85", "Patient_88","Patient_89"]

print("Organizing treat .nii.gz files into RM treamtnet folders...")

# Loop through patient folders
for patient_folder in os.listdir(base_path):
    if not patient_folder.startswith("Patient_"):
        continue
    if patient_folder in patients:
        patient = patient_folder.split("_")[1]
        ct_folder = os.path.join(base_path, patient_folder, f"P{patient}_CT_1_TRATAMENTO")
        rm_folder = os.path.join(base_path, patient_folder, f"P{patient}_RM_1_TRATAMENTO")
    
        if not os.path.exists(ct_folder):
            print(f"CT folder not found for Patient_{patient}, skipping.")
            continue
    
        if not os.path.exists(rm_folder):
            os.makedirs(rm_folder)
            print(f" Created RM folder for Patient_{patient}")
    
        for file in os.listdir(ct_folder):
            if file.endswith(".nii.gz") and "treat" in file:
                src_path = os.path.join(ct_folder, file)
                dst_path = os.path.join(rm_folder, file)
    
                try:
                    shutil.move(src_path, dst_path)
                    print(f"Moved: {file} → {rm_folder}")
                except Exception as e:
                    print(f" Error moving {file}: {e}")

print("✅ All done.")

Organizing treat .nii.gz files into RM treamtnet folders...
Moved: mask_Optic-Tract-Left_treat.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_69/P69_RM_1_TRATAMENTO
Moved: mask_Eye-Left_treat.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_69/P69_RM_1_TRATAMENTO
Moved: mask_PTV-temporal-1X20GY_treat.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_69/P69_RM_1_TRATAMENTO
Moved: mask_Lens-Left_treat.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_69/P69_RM_1_TRATAMENTO
Moved: mask_Optic-Nerve-Right_treat.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_69/P69_RM_1_TRATAMENTO
Moved: mask_CTV_treat.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_69/P69_RM_1_TRATAMENTO
Moved: mask_GTV-temporal_treat.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_69/P69_RM_1_TRATAMENTO
Moved: mask_Brain_treat.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Pat

### Resize Tumor Individual Masks to Follow_Up Images

In [15]:
print("Resampling Tumor INDIVIDUAL Masks to Follow_up Images")

base_path = "/notebooks/disk2/RT_Dataset_2/06_Processed_Patients"
mask_keywords = ["Whole-Brain", "Skull", "Outer-Contour"]
patients = ["Patient_54", "Patient_59","Patient_67", "Patient_68", "Patient_69", "Patient_70","Patient_82", "Patient_85", "Patient_88","Patient_89"]
        
# Loop through patient folders
for patient_folder in os.listdir(base_path):
    if not patient_folder.startswith("Patient_"):
        continue
    for patient_folder in patients:
        patient = patient_folder.split("_")[1]
        print(f"Processing Patient_{patient}")
    
        # Paths
        patient_path = os.path.join(base_path, f"Patient_{patient}")
        ct_folder = os.path.join(patient_path, f"P{patient}_CT_1_TRATAMENTO")
        follow_up_folder = os.path.join(patient_path, f"P{patient}_RM_1_FOLLOW_UP")
    
        # Find Follow-up image (one .nii.gz file inside treatment folder)
        mr_volume_image = None
        if os.path.exists(follow_up_folder):
            for file in os.listdir(follow_up_folder):
                if file.endswith(".nii.gz"):
                    mr_volume_image = sitk.ReadImage(os.path.join(follow_up_folder, file))
                    break
    
        if mr_volume_image is None:
            print(f"No follow up image found for Patient_{patient}, skipping.")
            continue
    
        tumors_path = ct_folder
    
        # Resample all tumor masks
        for file in os.listdir(ct_folder):
            if file.startswith("mask_") and file.endswith(".nii.gz") and not any(keyword in file for keyword in mask_keywords):
                mask_name = file.removesuffix(".nii.gz")
                tumor_mask_path = os.path.join(tumors_path, file)
                tumor_mask_treat_path = os.path.join(tumors_path, mask_name + "_treat_fu.nii.gz")
    
                tumor_mask = sitk.ReadImage(tumor_mask_path)
    
                # Resample to match MR volume
                resampler = sitk.ResampleImageFilter()
                resampler.SetReferenceImage(mr_volume_image)
                resampler.SetInterpolator(sitk.sitkNearestNeighbor)
                resampler.SetOutputOrigin(mr_volume_image.GetOrigin())
                resampler.SetOutputSpacing(mr_volume_image.GetSpacing())
                resampler.SetOutputDirection(mr_volume_image.GetDirection())
                resampler.SetSize(mr_volume_image.GetSize())

                resampled_mask = resampler.Execute(tumor_mask)
                sitk.WriteImage(resampled_mask, tumor_mask_treat_path)
    
                print(f"Resampled: {mask_name}")
    
        print(f"Done for Patient_{patient}")


Resampling Tumor INDIVIDUAL Masks to Follow_up Images
Processing Patient_54
Resampled: mask_Lens-Right
Resampled: mask_Chiasm
Resampled: mask_Optic-Nerve-Right
Resampled: mask_Brainstem
Resampled: mask_PTV-frontal-D-17Gy
Resampled: mask_Eye-Left
Resampled: mask_PTV-frontal-E-post-18Gy
Resampled: mask_Brain
Resampled: mask_GTV-frontal-E-Ant-18Gy
Resampled: mask_Eye-Right
Resampled: mask_PTV-frontal-E-anterior
Resampled: mask_Lens-Left
Resampled: mask_GTV-occiptal-E
Resampled: mask_Optic-Nerve-Left
Resampled: mask_PTV-occiptal-E-18GY
Resampled: mask_GTV-frontal-E-post
Done for Patient_54
Processing Patient_59
No follow up image found for Patient_59, skipping.
Processing Patient_67
Resampled: mask_PTV-18Gy-eval
Resampled: mask_Lens-Right
Resampled: mask_via-optica
Resampled: mask_PTV-frontal-E-5x6
Resampled: mask_Chiasm
Resampled: mask_GTV-parietal-E
Resampled: mask_PTV-frontal-D-18Gy
Resampled: mask_Optic-Nerve-Right
Resampled: mask_PTV-front-D-p-5x6
Resampled: mask_CTV-temp-E
Resampled:

RuntimeError: Exception thrown in SimpleITK ImageFileReader_Execute: /tmp/SimpleITK/Code/IO/src/sitkImageReaderBase.cxx:91:
sitk::ERROR: The file "/notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_54/P54_CT_1_TRATAMENTO/mask_Lens-Left_treat_fu_treat_fu_treat_fu_treat_fu.nii.gz" does not exist.

In [34]:
# Base path for patients
base_path = "/notebooks/disk2/RT_Dataset_2/06_Processed_Patients"

print("Organizing treat_fu .nii.gz files into RM follow_up folders...")

# Loop through patient folders
for patient_folder in os.listdir(base_path):
    if not patient_folder.startswith("Patient_"):
        continue

    patient = patient_folder.split("_")[1]
    ct_folder = os.path.join(base_path, patient_folder, f"P{patient}_CT_1_TRATAMENTO")
    rm_folder = os.path.join(base_path, patient_folder, f"P{patient}_RM_1_FOLLOW_UP")

    if not os.path.exists(ct_folder):
        print(f"CT folder not found for Patient_{patient}, skipping.")
        continue

    if not os.path.exists(rm_folder):
        os.makedirs(rm_folder)
        print(f" Created RM folder for Patient_{patient}")

    for file in os.listdir(ct_folder):
        if file.endswith(".nii.gz") and "treat_fu" in file:
            src_path = os.path.join(ct_folder, file)
            dst_path = os.path.join(rm_folder, file)

            try:
                shutil.move(src_path, dst_path)
                print(f"Moved: {file} → {rm_folder}")
            except Exception as e:
                print(f" Error moving {file}: {e}")

print("✅ All done.")

Organizing treat_fu .nii.gz files into RM treamtnet folders...
Moved: mask_Optic-Tract-Right_treat_fu.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_FOLLOW_UP
Moved: mask_Chiasm_treat_fu.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_FOLLOW_UP
Moved: mask_Optic-Nerve-Right_treat_fu.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_FOLLOW_UP
Moved: mask_Brain_treat_fu.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_FOLLOW_UP
Moved: mask_Lens-Left_treat_fu.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_FOLLOW_UP
Moved: mask_Eye-Right_treat_fu.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_FOLLOW_UP
Moved: mask_Brainstem_treat_fu.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_FOLLOW_UP
Moved: mask_Lens-Right_treat_fu.nii.gz → /notebooks/disk2/RT_Dataset_2/06_

### Resize Tumor Individual Masks to CT Images

In [35]:
print("Resampling Tumor INDIVIDUAL Masks to CT Images")

base_path = "/notebooks/disk2/RT_Dataset_2/06_Processed_Patients"
mask_keywords = ["Whole-Brain", "Skull", "Outer-Contour"]

# Loop through patient folders
for patient_folder in os.listdir(base_path):
    if not patient_folder.startswith("Patient_"):
        continue

    patient = patient_folder.split("_")[1]
    print(f"Processing Patient_{patient}")

    # Paths
    patient_path = os.path.join(base_path, f"Patient_{patient}")
    ct_folder = os.path.join(patient_path, f"P{patient}_CT_1_TRATAMENTO")

    # Find Follow-up image (one .nii.gz file inside treatment folder)
    ct_image = None
    if os.path.exists(ct_folder):
        for file in os.listdir(ct_folder):
            if file.endswith(".nii.gz") and "Denoised" in file:
                ct_image = sitk.ReadImage(os.path.join(ct_folder, file))
                break

    if ct_image is None:
        print(f"No CT image found for Patient_{patient}, skipping.")
        continue

    tumors_path = ct_folder

    # Resample all tumor masks
    for file in os.listdir(ct_folder):
        if file.startswith("mask_") and file.endswith(".nii.gz") and not any(keyword in file for keyword in mask_keywords):
            mask_name = file.removesuffix(".nii.gz")
            tumor_mask_path = os.path.join(tumors_path, file)
            tumor_mask_treat_path = os.path.join(tumors_path, mask_name + "_treat_ct.nii.gz")

            tumor_mask = sitk.ReadImage(tumor_mask_path)

            # Resample to match CT volume
            resampler = sitk.ResampleImageFilter()
            resampler.SetReferenceImage(ct_image)
            resampler.SetInterpolator(sitk.sitkNearestNeighbor)
            resampler.SetOutputOrigin(ct_image.GetOrigin())
            resampler.SetOutputSpacing(ct_image.GetSpacing())
            resampler.SetOutputDirection(ct_image.GetDirection())
            resampler.SetSize(ct_image.GetSize())

            resampled_mask = resampler.Execute(tumor_mask)
            sitk.WriteImage(resampled_mask, tumor_mask_treat_path)

            print(f"Resampled: {mask_name}")

    print(f"Done for Patient_{patient}")


Resampling Tumor INDIVIDUAL Masks to CT Images
Processing Patient_61
Resampled: mask_Lens-Right
Resampled: mask_Chiasm
Resampled: mask_PTV-3x8
Resampled: mask_PTV-3x8-1cm
Resampled: mask_Optic-Nerve-Right
Resampled: mask_Brainstem
Resampled: mask_Eye-Left
Resampled: mask_Tumor
Resampled: mask_Optic-Tract-Right
Resampled: mask_Brain
Resampled: mask_Eye-Right
Resampled: mask_Lens-Left
Resampled: mask_PTV-3x8-opt
Resampled: mask_Optic-Tract-Left
Resampled: mask_Optic-Nerve-Left
Done for Patient_61
Processing Patient_78
Resampled: mask_MtFrontEProf
Resampled: mask_Lens-Right
Resampled: mask_MetVerCerebelar
Resampled: mask_MerFrEsq
Resampled: mask_MetFDirInf
Resampled: mask_MetODir
Resampled: mask_MtFrMedAntEs
Resampled: mask_Chiasm
Resampled: mask_MetCapIntDir
Resampled: mask_MetIntHemisf
Resampled: mask_MetFPDIR
Resampled: mask_MetParProfD
Resampled: mask_Optic-Nerve-Right
Resampled: mask_MetOccEsqInf
Resampled: mask_MetMesencDir
Resampled: mask_Brainstem
Resampled: mask_Eye-Left
Resample

In [44]:
print("Resampling Tumor INDIVIDUAL Masks to CT Images for patient 56")

base_path = "/notebooks/disk2/RT_Dataset_2/06_Processed_Patients"
mask_keywords = ["Whole-Brain", "Skull", "Outer-Contour"]

# Loop through patient folders
for patient_folder in os.listdir(base_path):
    if not patient_folder.startswith("Patient_"):
        continue

    patient = patient_folder.split("_")[1]
    if "56" in patient:

        # Paths
        patient_path = os.path.join(base_path, f"Patient_{patient}")
        ct_folder = os.path.join(patient_path, f"P{patient}_CT_1_TRATAMENTO")
    
        # Find Follow-up image (one .nii.gz file inside treatment folder)
        ct_image = None
        if os.path.exists(ct_folder):
            for file in os.listdir(ct_folder):
                if file.endswith(".nii.gz") and "Windowed" in file:
                    ct_image = sitk.ReadImage(os.path.join(ct_folder, file))
                    break
    
        if ct_image is None:
            print(f"No CT image found for Patient_{patient}, skipping.")
            continue
    
        tumors_path = ct_folder
    
        # Resample all tumor masks
        for file in os.listdir(ct_folder):
            if file.startswith("mask_") and file.endswith(".nii.gz") and not any(keyword in file for keyword in mask_keywords):
                mask_name = file.removesuffix(".nii.gz")
                tumor_mask_path = os.path.join(tumors_path, file)
                tumor_mask_treat_path = os.path.join(tumors_path, mask_name + "_treat_ct.nii.gz")
    
                tumor_mask = sitk.ReadImage(tumor_mask_path)
    
                # Resample to match CT volume
                resampler = sitk.ResampleImageFilter()
                resampler.SetReferenceImage(ct_image)
                resampler.SetInterpolator(sitk.sitkNearestNeighbor)
                resampler.SetOutputOrigin(ct_image.GetOrigin())
                resampler.SetOutputSpacing(ct_image.GetSpacing())
                resampler.SetOutputDirection(ct_image.GetDirection())
                resampler.SetSize(ct_image.GetSize())
    
                resampled_mask = resampler.Execute(tumor_mask)
                sitk.WriteImage(resampled_mask, tumor_mask_treat_path)
    
                print(f"Resampled: {mask_name}")
    
        print(f"Done for Patient_{patient}")

Resampling Tumor INDIVIDUAL Masks to CT Images for patient 56
Resampled: mask_Lens-Right
Resampled: mask_BL-ICT-Frameless-Extension-011
Resampled: mask_Pituitary-Gland
Resampled: mask_Chiasm
Resampled: mask_2-GTV-met-cerebelo
Resampled: mask_Hypothalamus
Resampled: mask_Optic-Nerve-Right
Resampled: mask_1-GTV-Met
Resampled: mask_Coord-1
Resampled: mask_Brainstem
Resampled: mask_Pineal-Gland
Resampled: mask_Eye-Left
Resampled: mask_Ventricles
Resampled: mask_2-GTV-met-cerebelo-10-mm
Resampled: mask_Cochlea-Left
Resampled: mask_Optic-Tract-Right
Resampled: mask_Cochlea-Right
Resampled: mask_Eye-Right
Resampled: mask_Cerebellum
Resampled: mask_1-GTV-Met-10-mm
Resampled: mask_Lens-Left
Resampled: mask_Cerebrum
Resampled: mask_Optic-Tract-Left
Resampled: mask_Cerebro-SANO
Resampled: mask_BL-ICT-Frameless-Extension-170
Resampled: mask_Optic-Nerve-Left
Done for Patient_56


### Resize Isodose Volume Masks to Treatment Images

In [37]:
print("Resampling Isodose Volume Masks to Treatment Images")

base_path = "/notebooks/disk2/RT_Dataset_2/06_Processed_Patients"

# Loop through patient folders
for patient_folder in os.listdir(base_path):
    if not patient_folder.startswith("Patient_"):
        continue

    patient = patient_folder.split("_")[1]
    print(f"Processing Patient_{patient}")

    # Define paths
    patient_path = os.path.join(base_path, f"Patient_{patient}")
    ct_folder = os.path.join(patient_path, f"P{patient}_CT_1_TRATAMENTO")
    treatment_folder = os.path.join(patient_path, f"P{patient}_RM_1_TRATAMENTO")

    # Find treatment reference image
    treatment_volume_image = None
    if os.path.exists(treatment_folder):
        for file in os.listdir(treatment_folder):
            if file.endswith("nii.gz"):
                treatment_volume_image = sitk.ReadImage(os.path.join(treatment_folder, file))
                break

    if treatment_volume_image is None:
        print(f"No treatment image found for Patient_{patient}, skipping.")
        continue

    # Loop through Treatment folder to find isodose masks
    for file in os.listdir(ct_folder):
        if "Resampled_Filtered" in file and file.endswith(".nii.gz"):
            mask_name = file.removesuffix(".nii.gz")
            isodose_mask_path = os.path.join(ct_folder, file)
            isodose_mask_treat_path = os.path.join(ct_folder, mask_name + "_treat.nii.gz")

            try:
                isodose_mask = sitk.ReadImage(isodose_mask_path)

                # Resample to match treatment image
                resampler = sitk.ResampleImageFilter()
                resampler.SetReferenceImage(treatment_volume_image)
                resampler.SetInterpolator(sitk.sitkNearestNeighbor)
                resampler.SetOutputOrigin(treatment_volume_image.GetOrigin())
                resampler.SetOutputSpacing(treatment_volume_image.GetSpacing())
                resampler.SetOutputDirection(treatment_volume_image.GetDirection())
                resampler.SetSize(treatment_volume_image.GetSize())

                resampled_mask = resampler.Execute(isodose_mask)
                sitk.WriteImage(resampled_mask, isodose_mask_treat_path)

                print(f"Resampled mask: {mask_name}")

            except Exception as e:
                print(f"Error processing {file}: {e}")

    print(f"✅ Done for Patient_{patient}")


Resampling Isodose Volume Masks to Treatment Images

Processing Patient_61
Resampled mask: P61_Isodose_Volume_mask_PTV-3x8-1cm_Resampled_Filtered
Resampled mask: P61_Isodose_Volume_mask_PTV-3x8-opt_Resampled_Filtered
Resampled mask: P61_Isodose_Volume_mask_PTV-3x8_Resampled_Filtered
✅ Done for Patient_61

Processing Patient_78
Resampled mask: P78_Isodose_Volume_mask_MetOccDir2_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetODir2_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetHCEs_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetODir3_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetHCDir_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetTempPostE_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetODir_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetOccEsqInf_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetOccE2Post_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetParD

In [38]:
# Base path for patients
base_path = "/notebooks/disk2/RT_Dataset_2/06_Processed_Patients"

print("Organizing resampled_filtered_treat .nii.gz files into RM treamtnet folders...")

# Loop through patient folders
for patient_folder in os.listdir(base_path):
    if not patient_folder.startswith("Patient_"):
        continue

    patient = patient_folder.split("_")[1]
    ct_folder = os.path.join(base_path, patient_folder, f"P{patient}_CT_1_TRATAMENTO")
    rm_folder = os.path.join(base_path, patient_folder, f"P{patient}_RM_1_TRATAMENTO")

    if not os.path.exists(ct_folder):
        print(f"CT folder not found for Patient_{patient}, skipping.")
        continue

    if not os.path.exists(rm_folder):
        os.makedirs(rm_folder)
        print(f" Created RM folder for Patient_{patient}")

    for file in os.listdir(ct_folder):
        if file.endswith("Resampled_Filtered_treat.nii.gz"):
            src_path = os.path.join(ct_folder, file)
            dst_path = os.path.join(rm_folder, file)

            try:
                shutil.move(src_path, dst_path)
                print(f"Moved: {file} → {rm_folder}")
            except Exception as e:
                print(f" Error moving {file}: {e}")

print("✅ All done.")

Organizing resampled_filtered_treat .nii.gz files into RM treamtnet folders...
Moved: P61_Isodose_Volume_mask_PTV-3x8_Resampled_Filtered_treat.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_TRATAMENTO
Moved: P61_Isodose_Volume_mask_PTV-3x8-1cm_Resampled_Filtered_treat.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_TRATAMENTO
Moved: P61_Isodose_Volume_mask_PTV-3x8-opt_Resampled_Filtered_treat.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_TRATAMENTO
Moved: P78_Isodose_Volume_mask_MetHCEs_Resampled_Filtered_treat.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_78/P78_RM_1_TRATAMENTO
Moved: P78_Isodose_Volume_mask_MetFEsq_Resampled_Filtered_treat.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_78/P78_RM_1_TRATAMENTO
Moved: P78_Isodose_Volume_mask_MetOccDir2_Resampled_Filtered_treat.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patie

### Resize Isodose Volume Masks to Follow_Up Images

In [39]:
print("Resampling Isodose Volume Masks to Follow_up Images")

base_path = "/notebooks/disk2/RT_Dataset_2/06_Processed_Patients"

# Loop through patient folders
for patient_folder in os.listdir(base_path):
    if not patient_folder.startswith("Patient_"):
        continue

    patient = patient_folder.split("_")[1]
    print(f"Processing Patient_{patient}")

    # Define paths
    patient_path = os.path.join(base_path, f"Patient_{patient}")
    ct_folder = os.path.join(patient_path, f"P{patient}_CT_1_TRATAMENTO")
    follow_up_folder = os.path.join(patient_path, f"P{patient}_RM_1_FOLLOW_UP")

    # Find treatment reference image
    follow_up_volume_image = None
    if os.path.exists(follow_up_folder):
        for file in os.listdir(follow_up_folder):
            if file.endswith("nii.gz"):
                follow_up_volume_image = sitk.ReadImage(os.path.join(follow_up_folder, file))
                break

    if follow_up_volume_image is None:
        print(f"No treatment image found for Patient_{patient}, skipping.")
        continue

    # Loop through Treatment folder to find isodose masks
    for file in os.listdir(ct_folder):
        if "Resampled_Filtered" in file and file.endswith(".nii.gz"):
            mask_name = file.removesuffix(".nii.gz")
            isodose_mask_path = os.path.join(ct_folder, file)
            isodose_mask_treat_path = os.path.join(ct_folder, mask_name + "_treat_fu.nii.gz")

            try:
                isodose_mask = sitk.ReadImage(isodose_mask_path)

                # Resample to match treatment image
                resampler = sitk.ResampleImageFilter()
                resampler.SetReferenceImage(follow_up_volume_image)
                resampler.SetInterpolator(sitk.sitkNearestNeighbor)
                resampler.SetOutputOrigin(follow_up_volume_image.GetOrigin())
                resampler.SetOutputSpacing(follow_up_volume_image.GetSpacing())
                resampler.SetOutputDirection(follow_up_volume_image.GetDirection())
                resampler.SetSize(follow_up_volume_image.GetSize())

                resampled_mask = resampler.Execute(isodose_mask)
                sitk.WriteImage(resampled_mask, isodose_mask_treat_path)

                print(f"Resampled mask: {mask_name}")

            except Exception as e:
                print(f"Error processing {file}: {e}")

    print(f"✅ Done for Patient_{patient}")


Resampling Isodose Volume Masks to Follow_up Images
Processing Patient_61
Resampled mask: P61_Isodose_Volume_mask_PTV-3x8-1cm_Resampled_Filtered
Resampled mask: P61_Isodose_Volume_mask_PTV-3x8-opt_Resampled_Filtered
Resampled mask: P61_Isodose_Volume_mask_PTV-3x8_Resampled_Filtered
✅ Done for Patient_61
Processing Patient_78
Resampled mask: P78_Isodose_Volume_mask_MetOccDir2_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetODir2_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetHCEs_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetODir3_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetHCDir_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetTempPostE_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetODir_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetOccEsqInf_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetOccE2Post_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetParDir

In [40]:
base_path = "/notebooks/disk2/RT_Dataset_2/06_Processed_Patients"

print("Organizing resampled_filtered_treat .nii.gz files into RM follow_up folders...")

# Loop through patient folders
for patient_folder in os.listdir(base_path):
    if not patient_folder.startswith("Patient_"):
        continue

    patient = patient_folder.split("_")[1]
    ct_folder = os.path.join(base_path, patient_folder, f"P{patient}_CT_1_TRATAMENTO")
    rm_folder = os.path.join(base_path, patient_folder, f"P{patient}_RM_1_FOLLOW_UP")

    if not os.path.exists(ct_folder):
        print(f"CT folder not found for Patient_{patient}, skipping.")
        continue

    if not os.path.exists(rm_folder):
        os.makedirs(rm_folder)
        print(f" Created RM folder for Patient_{patient}")

    for file in os.listdir(ct_folder):
        if file.endswith("Resampled_Filtered_treat_fu.nii.gz"):
            src_path = os.path.join(ct_folder, file)
            dst_path = os.path.join(rm_folder, file)

            try:
                shutil.move(src_path, dst_path)
                print(f"Moved: {file} → {rm_folder}")
            except Exception as e:
                print(f" Error moving {file}: {e}")

print("✅ All done.")

Organizing resampled_filtered_treat .nii.gz files into RM follow_up folders...
Moved: P61_Isodose_Volume_mask_PTV-3x8-opt_Resampled_Filtered_treat_fu.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_FOLLOW_UP
Moved: P61_Isodose_Volume_mask_PTV-3x8-1cm_Resampled_Filtered_treat_fu.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_FOLLOW_UP
Moved: P61_Isodose_Volume_mask_PTV-3x8_Resampled_Filtered_treat_fu.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_61/P61_RM_1_FOLLOW_UP
Moved: P78_Isodose_Volume_mask_MetOccEsqInf_Resampled_Filtered_treat_fu.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_78/P78_RM_1_FOLLOW_UP
Moved: P78_Isodose_Volume_mask_MetHCDir_Resampled_Filtered_treat_fu.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Processed_Patients/Patient_78/P78_RM_1_FOLLOW_UP
Moved: P78_Isodose_Volume_mask_MetOccE2Post_Resampled_Filtered_treat_fu.nii.gz → /notebooks/disk2/RT_Dataset_2/06_Pro

### Resize Isodose Volume Masks to CT Images

In [42]:
print("Resampling Isodose Volume Masks to CT Images")

base_path = "/notebooks/disk2/RT_Dataset_2/06_Processed_Patients"

# Loop through patient folders
for patient_folder in os.listdir(base_path):
    if not patient_folder.startswith("Patient_"):
        continue

    patient = patient_folder.split("_")[1]
    print(f"Processing Patient_{patient}")

    # Define paths
    patient_path = os.path.join(base_path, f"Patient_{patient}")
    ct_folder = os.path.join(patient_path, f"P{patient}_CT_1_TRATAMENTO")

    # Find treatment reference image
    ct_image = None
    if os.path.exists(ct_folder):
        for file in os.listdir(ct_folder):
            if file.endswith("nii.gz") and "Denoised" in file:
                ct_image = sitk.ReadImage(os.path.join(ct_folder, file))
                break

    if ct_image is None:
        print(f"No treatment image found for Patient_{patient}, skipping.")
        continue

    # Loop through Treatment folder to find isodose masks
    for file in os.listdir(ct_folder):
        if "Resampled_Filtered" in file and file.endswith(".nii.gz"):
            mask_name = file.removesuffix(".nii.gz")
            isodose_mask_path = os.path.join(ct_folder, file)
            isodose_mask_treat_path = os.path.join(ct_folder, mask_name + "_treat_ct.nii.gz")

            try:
                isodose_mask = sitk.ReadImage(isodose_mask_path)

                # Resample to match treatment image
                resampler = sitk.ResampleImageFilter()
                resampler.SetReferenceImage(ct_image)
                resampler.SetInterpolator(sitk.sitkNearestNeighbor)
                resampler.SetOutputOrigin(ct_image.GetOrigin())
                resampler.SetOutputSpacing(ct_image.GetSpacing())
                resampler.SetOutputDirection(ct_image.GetDirection())
                resampler.SetSize(ct_image.GetSize())

                resampled_mask = resampler.Execute(isodose_mask)
                sitk.WriteImage(resampled_mask, isodose_mask_treat_path)

                print(f"Resampled mask: {mask_name}")

            except Exception as e:
                print(f"Error processing {file}: {e}")

    print(f"✅ Done for Patient_{patient}")


Resampling Isodose Volume Masks to Follow_up Images
Processing Patient_61
Resampled mask: P61_Isodose_Volume_mask_PTV-3x8-1cm_Resampled_Filtered
Resampled mask: P61_Isodose_Volume_mask_PTV-3x8-opt_Resampled_Filtered
Resampled mask: P61_Isodose_Volume_mask_PTV-3x8_Resampled_Filtered
✅ Done for Patient_61
Processing Patient_78
Resampled mask: P78_Isodose_Volume_mask_MetOccDir2_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetODir2_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetHCEs_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetODir3_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetHCDir_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetTempPostE_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetODir_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetOccEsqInf_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetOccE2Post_Resampled_Filtered
Resampled mask: P78_Isodose_Volume_mask_MetParDir

In [43]:
print("Resampling Isodose Volume Masks to CT Images just for patient 56")

base_path = "/notebooks/disk2/RT_Dataset_2/06_Processed_Patients"

# Loop through patient folders
for patient_folder in os.listdir(base_path):
    if not patient_folder.startswith("Patient_"):
        continue
    
    patient = patient_folder.split("_")[1]
    if "56" in patient:
        # Define paths
        patient_path = os.path.join(base_path, f"Patient_{patient}")
        ct_folder = os.path.join(patient_path, f"P{patient}_CT_1_TRATAMENTO")
    
        # Find treatment reference image
        ct_image = None
        if os.path.exists(ct_folder):
            for file in os.listdir(ct_folder):
                if file.endswith("nii.gz") and "Windowed" in file:
                    ct_image = sitk.ReadImage(os.path.join(ct_folder, file))
                    break
    
        if ct_image is None:
            print(f"No treatment image found for Patient_{patient}, skipping.")
            continue
    
        # Loop through Treatment folder to find isodose masks
        for file in os.listdir(ct_folder):
            if "Resampled_Filtered" in file and file.endswith(".nii.gz"):
                mask_name = file.removesuffix(".nii.gz")
                isodose_mask_path = os.path.join(ct_folder, file)
                isodose_mask_treat_path = os.path.join(ct_folder, mask_name + "_treat_ct.nii.gz")
    
                try:
                    isodose_mask = sitk.ReadImage(isodose_mask_path)
    
                    # Resample to match treatment image
                    resampler = sitk.ResampleImageFilter()
                    resampler.SetReferenceImage(ct_image)
                    resampler.SetInterpolator(sitk.sitkNearestNeighbor)
                    resampler.SetOutputOrigin(ct_image.GetOrigin())
                    resampler.SetOutputSpacing(ct_image.GetSpacing())
                    resampler.SetOutputDirection(ct_image.GetDirection())
                    resampler.SetSize(ct_image.GetSize())
    
                    resampled_mask = resampler.Execute(isodose_mask)
                    sitk.WriteImage(resampled_mask, isodose_mask_treat_path)
    
                    print(f"Resampled mask: {mask_name}")
    
                except Exception as e:
                    print(f"Error processing {file}: {e}")
    
        print(f"✅ Done for Patient_{patient}")


Resampling Isodose Volume Masks to CT Images just for patient 56
Resampled mask: P56_Isodose_Volume_mask_2-GTV-met-cerebelo-10-mm_Resampled_Filtered
Resampled mask: P56_Isodose_Volume_mask_1-GTV-Met_Resampled_Filtered
Resampled mask: P56_Isodose_Volume_mask_1-GTV-Met-10-mm_Resampled_Filtered
Resampled mask: P56_Isodose_Volume_mask_2-GTV-met-cerebelo_Resampled_Filtered
✅ Done for Patient_56
